# コンテキストを含むプロンプトを使用する LangChainに よる Bedrock

> *この ノートブック は SageMaker Studio の **`Data Science 3.0`** カーネルをご利用ください*

## 導入

このノートブックでは、カスタマーサポートエンジニアから受けたカスタマーサービスの質に満足していない顧客へのメール返信を生成する方法を説明します。不満を抱いている顧客から受信した実際のメールの内容を提供することで、モデルに追加のコンテキストを提供します。

プロンプトにはコンテキストが追加されているため、このノートブックの Amazon Titan Text Large モデルによって生成されたテキストは、以前に Zero-Shot プロンプトで作成されたコンテンツよりも品質と関連性がはるかに優れています。

[LangChain](https://python.langchain.com/docs/get_started/introduction.html) は、言語モデルを利用したアプリケーションを開発するためのフレームワークです。このフレームワークの主な特徴により、さまざまなコンポーネントを連結して高度なユースケースを作成することで、大規模言語モデルを拡張できます。

このノートブックでは、LangChain が提供する Bedrock API を使用します。この例で使用されているプロンプトは、テキスト生成リクエストにコンテキストを追加するためのカスタム LangChain プロンプトテンプレートを作成します。 

**Note:** *このノートブックは AWS 内外問わず実行することができます*

#### コンテキスト
先ほどの `01_zero_shot_generation.ipynb` の例では、LangChain フレームワークを使用して Amazon Bedrock API と通信する方法について説明しました。このノートブックでは、`PromptTemplate` を使用して、LangChain フレームワークを同様のユースケースで利用できるようにもう少し複雑さを加えようとします。`PromptTemplate` を使うと、後で情報を入力できる汎用シェルを作成したり、さまざまなシナリオに基づいてモデル出力を取得したりできます。

このノートブックの中で、LangChain フレームワーク内の Amazon Bedrock 統合の使用方法と、それを使用して `PromptTemplate` を使用してテキストを生成する方法を探ります。

#### パターン
Amazon Bedrock API の LangChain 実装に、タスク、命令、入力を提供するだけで、追加の例は示さずに出力を生成できます。ここでの目的は、強力な LLM が目の前のタスクをいかに簡単に理解し、説得力のある出力を生成するかを示すことです。

![](./images/bedrock_langchain.jpg)

#### ユースケース
Amazon Bedrock のモデルの生成機能を実証するために、E メール生成のユースケースを見てみましょう。

#### ペルソナ
あなたは AnyCompany のカスタマーサービスマネージャーである Bob ですが、一部の顧客はカスタマーサービスに満足しておらず、カスタマーサポートエンジニアが提供するサービスに否定的なフィードバックをしています。今度は、サービスの質が悪かったことを謙虚に謝罪する顧客に応え、信頼を取り戻したいと考えています。以前のメールでのやり取りから得た顧客の感情に合わせて、ヒューマンフレンドリーなメールを大量に生成するには、LLM の助けが必要です。

#### 実装
このユースケースを実現するために、顧客の以前のメールに基づいてサンキューノート付きのメールを生成する方法を紹介します。ここでは Amazon Bedrock LangChain インテグレーションを使用する Amazon Titan Text Large モデルを使用します。 


## セットアップ

このノートブックの以降を実行する前に、以下のセルを実行して (必要なライブラリがインストールされていることを確認し) Bedrockに接続する必要があります。

セットアップの仕組みと ⚠️ **変更の要否** の詳細については [Bedrock boto3 setup notebook](../00_Intro/bedrock_boto3_setup.ipynb) を参照ください。

このノートブックには、入力プロンプト内のトークンの数をカウントするために使用する [Hugging Face Transformers](https://huggingface.co/docs/transformers/index) ライブラリもインストールします。

In [ ]:
# 事前にリポジトリルートから `download-dependencies.sh` を実行済みであることを確認してください!
%pip install --no-build-isolation --force-reinstall \
    ../dependencies/awscli-*-py3-none-any.whl \
    ../dependencies/boto3-*-py3-none-any.whl \
    ../dependencies/botocore-*-py3-none-any.whl

%pip install --quiet langchain==0.0.249 "transformers>=4.24,<5"

In [ ]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ 必要に応じて AWS 設定に関する以下のコードのコメントを解除、編集してください ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

## LangChain インテグレーションを使用して Bedrock クライアントを起動

llms から Bedrock クラスのインスタンスを作成することから始めましょう。これは Amazon Bedrock で利用できるモデルの `model_id` を想定しています。 

オプションで、以前に作成した boto3 の `client` のほか、`temperature`、`topP`、`maxTokenCount`、`stopSequences` などのパラメータを保持できる `model_kwargs` を渡すことができます (パラメータの詳細については Amazon Bedrock コンソールで確認できます)。

Amazon Bedrock で利用可能なテキスト生成モデルには、次の ID があります。

- `amazon.titan-tg1-large`
- `ai21.j2-grande-instruct`
- `ai21.j2-jumbo-instruct`
- `anthropic.claude-instant-v1`
- `anthropic.claude-v1`

`model_kwargs` はモデルが異なればサポートも異なることに注意してください。

In [ ]:
from langchain.llms.bedrock import Bedrock

inference_modifier = {'max_tokens_to_sample':4096, 
                      "temperature":0.5,
                      "top_k":250,
                      "top_p":1,
                      "stop_sequences": ["\n\nHuman"]
                     }

textgen_llm = Bedrock(model_id = "anthropic.claude-v1",
                    client = boto3_bedrock, 
                    model_kwargs = inference_modifier 
                    )


## LangChain カスタムプロンプトテンプレートを作成

プロンプトのテンプレートを作成することで、実行ごとに異なる入力変数を渡すことができます。これは、データベースから取得するさまざまな入力変数を使用してコンテンツを生成する必要がある場合に便利です。

以前のラボではプロンプトをハードコーディングしていました。ここでは、複数の顧客が同様の否定的なフィードバックを送信していて、それらの顧客の各メールを使用して謝罪を返信したい場合であり、応答も少しパーソナライズしておきたい場合を想定します。次のセルでは、このパターンを実現するための `PromptTemplate` を作成する方法を検討しています。

In [ ]:
from langchain import PromptTemplate

# 複数の入力変数を含むプロンプトテンプレートの作成
multi_var_prompt = PromptTemplate(
    input_variables=["customerServiceManager", "customerName", "feedbackFromCustomer"], 
    template="""Create an apology email from the Service Manager {customerServiceManager} to {customerName}. 
   in response to the following feedback that was received from the customer: {feedbackFromCustomer}.
   """
)

# 入力変数に値を渡す
prompt = multi_var_prompt.format(customerServiceManager="Bob", 
                                 customerName="John Doe", 
                                 feedbackFromCustomer="""Hello Bob,
     I am very disappointed with the recent experience I had when I called your customer support.
     I was expecting an immediate call back but it took three days for us to get a call back.
     The first suggestion to fix the problem was incorrect. Ultimately the problem was fixed after three days.
     We are very unhappy with the response provided and may consider taking our business elsewhere.
     """
     )


In [ ]:
num_tokens = textgen_llm.get_num_tokens(prompt)
print(f"Our prompt has {num_tokens} tokens")

## 再呼び出し

プロンプトテンプレートを使用して起動すると、精選された応答が返されることが期待されます。

In [ ]:
response = textgen_llm(prompt)

email = response[response.index('\n')+1:]

print_ww(email)

## 結論

結論として、[Zero-Shot](./01_zero_shot_generation.ja.ipynb) プロンプトによりコンテキストなしで LLM を呼び出しても、望ましい結果が得られない可能性があることがわかりました。コンテキストを追加し、さらに prompt テンプレートを使用して LLM からの出力を制限することで、目的の出力を正常に得ることができます。